# This code help you to get graph from VK social network

In [27]:
import vk
import pandas as pd
import requests
import random
from time import time

session = vk.Session(access_token='you secret token')

# session = vk.Session()
api = vk.API(session, lang='en', timeout=20, v='5.62')

#### Util functions

In [41]:
def getUserInfo(userID, personalInfo = False, friendslist = False):

    resultDict = {}
    wallPosts = []
    friends = []
    full = 0
    personal = []
    if personalInfo:
        try:
#             https://vk.com/dev/fields
            personal = api.users.get(user_ids=userID, fields = 'sex, country, city, connections, screen_name, bdate')
        except Exception as e:
            print("error in getting personal info")
            return
            
    uid = personal[0]['id']
    if friendslist and uid: 
        try:
            friends = api.friends.get(user_id = uid)
            print(len(friends))
        except Exception as e:
            print(e)
            print("could't get friend list")

    resultDict = {'id': uid, 'personal': personal, 'friends':friends}
   
    return resultDict



def get_friends_ids(user_id):
    friends_url = 'https://api.vk.com/method/friends.get?user_id={}' 
    json_response = requests.get(friends_url.format(user_id)).json()
    if json_response.get('error'):
#         print (json_response.get('error'))
        return list()
    return json_response[u'response']

In [56]:
g = getUserInfo('alina.sirazetdinova ', True, False)
print(g)

{'id': 110195469, 'friends': [], 'personal': [{'bdate': '15.11.1992', 'instagram': 'sirazinka', 'sex': 1, 'first_name': 'Alina', 'country': {'id': 1, 'title': 'Russia'}, 'twitter': 'Sirazinka', 'id': 110195469, 'last_name': 'Sirazetdinova', 'city': {'id': 60, 'title': 'Kazan'}, 'screen_name': 'alina.sirazetdinova'}]}


In [ ]:
def getWallPosts(userID):
    post_count = 0
    api.wall.get(owner_id = userID)
    temp_counter = post_count 
#     ostatok = post_count% 100
    my_range = 0
    
      
    myCount = 100 #количество постов, получаемых за 1 запрос
    myOffset = 0 #начальное смещение
   
    try:
        post_count = api.wall.get(owner_id = userID)[0] #количество постов в группе или профиле
    except Exception as e:
        print('не получается узнать сколько постов')
    temp_counter = post_count
    
                         
    if temp_counter > 200:#ограничиваю, чтобы брать только последние 200 постов, а не все посты со стены
        temp_counter = 200

    dd = []
    while myOffset < temp_counter:  #это означает меньше 200 или меньше
        r = api.wall.get(owner_id = userID, count = myCount, offset = myOffset)

        if temp_counter < 100:
            my_range = temp_counter
        else:
            my_range = 100
        
#         print(my_range)
        for i in range(1, my_range + 1):
            ownerOfPost = 0  
            try:
                ownerOfPost = r[i]['copy_owner_id']#
            except Exception as e:
                u = 0
#                 print('не получается узнать кто автор')
            post_text = r[i]['text']
    
#             print(post_text,'\n')
            
            dd.append(ownerOfPost)
            dd.append(post_text)
        myOffset = myOffset + 100
        temp_counter = temp_counter - 100
    return dd

In [ ]:
# numberOfUsers = 10000

def getManyUsers(numberOfUsers):
    """get random users """
    users = []
    counter = 1
    proc = 0.05
    maxUserId = 99999999

    t0 = time()

    for iteration in range(1, numberOfUsers):
    #     print('maxUser:', maxUserId)
        if counter % (numberOfUsers*proc) == 0:
            print ('percentage', iteration/float(numberOfUsers)*100)
            print("done in %0.3fs." % (time() - t0))
            maxUserId = int(maxUserId/10)
        if maxUserId <= 100:
            maxUserId = 182742236

        while  True:
            randomId = random.randrange(1, maxUserId)
            u = getUserInfo(randomId)
            if u != []:
#                 print("em", randomId)
                break
        global dfNew
        dfNew = dfNew.append(u, ignore_index=True)
#         appendDFToCSV_void(df,'vkCSV.csv')

#         df.to_csv('vkCSV.csv', header = True, mode = 'w',
#                   columns = ('id', 'personal', 'friends', 'wallPosts'), encoding='utf-8')
        users.append(u)
        counter += 1

    print ("100%")
    print("done in %0.3fs." % (time() - t0))
    print ("finish")
    return users

In [4]:
from queue import Queue
from pdb import set_trace
from tqdm import tqdm_notebook
def getGraph(userIds, depth = 3):
    """
    userIds might List 
    """
#     set_trace()
    result = {}
    q = Queue()
    for s in userIds:
        q.put(s)
    for d in range(depth):
        sz = q.qsize()
        for i in tqdm_notebook(range(sz)):
            u = q.get()
            info = getUserInfo(u, friendslist=True, personalInfo=False, wallposts=False)
            result[u] = info['friends']
            for v in result[u]:
                q.put(v)
    return result

In [6]:
g = getGraph([1], 2)

In [8]:
len(g)

725

### GetMutual friends

In [98]:
p = api.execute.getMutual(source='104277713', targets='12133829,')
print(p)
# 94665415 bl3000
# 12133829 Guzel
# darya_borisovna
# chuprin_arkady 104277713
# raimtriger

[[33827906, 47760473, 62823257, 95084466]]


### Get list of Kazan peoples
- top 1000 from city Kazan_id = 60
- [https://vk.com/int_kazan]  107840
- [https://vk.com/kazan.club] 85479
- [https://vk.com/kzngot]     51993

In [25]:
# top 1000
def getFromCity(gid=60, offset = 0):
    uids = api.execute.getFromCity(city_id = gid, ofs = offset)
    return uids
    
p = getFromCity(offset=0)
print('len p = ', len(p))
print(p)

len p =  985
[317079389, 13232402, 1533999, 170753080, 5390429, 133382104, 109324229, 26884310, 241255335, 38355695, 16409252, 5256711, 125336201, 13177579, 51546069, 102903659, 40217940, 131979462, 3436666, 172699321, 49637204, 146882269, 209868387, 25802046, 4420023, 93189739, 11559837, 136075314, 54799928, 31906342, 4116774, 176011311, 18419989, 4448851, 78167646, 12916234, 222981129, 12397503, 147935311, 8283125, 56574485, 59492651, 31218137, 105022081, 20082936, 1742750, 59099234, 136746949, 17281723, 84193583, 67799542, 156768849, 7408349, 6698778, 334729115, 6037655, 26273975, 85589540, 8290939, 6825515, 7442813, 40036266, 45422243, 127336993, 3100916, 91268742, 34635207, 37762454, 2345259, 44165266, 44983735, 12527415, 233202447, 282420896, 18790068, 5448563, 309726968, 146000605, 55028961, 176388170, 72191482, 256017426, 19885991, 50877229, 1387915, 7232748, 22187694, 292557829, 33367904, 18368326, 145106339, 140209774, 26385921, 18400203, 316297655, 80970099, 170477609, 14011

In [28]:
# get users from group
def getFromGroup(gid, offset = 0):
    uids = api.execute.getFromGroup(group_id = gid, ofs = offset)
    return uids

In [29]:
# [https://vk.com/int_kazan] 107869
p = getFromGroup(gid='int_kazan', offset = 100000)
print('len p = ', len(p))
print(p)

len p =  7184
[339437917, 339438551, 339443638, 339444319, 339447670, 339451356, 339452944, 339456789, 339456826, 339465818, 339467337, 339467951, 339471702, 339472104, 339472971, 339486531, 339487502, 339499166, 339502891, 339505470, 339505494, 339507750, 339510847, 339512753, 339513628, 339519084, 339519457, 339520234, 339531958, 339538373, 339538409, 339539238, 339540851, 339542308, 339552102, 339569901, 339573419, 339579305, 339589942, 339596930, 339600305, 339603923, 339620794, 339635217, 339642906, 339643735, 339644107, 339646361, 339657096, 339657330, 339657423, 339657790, 339663046, 339676383, 339676565, 339678127, 339686708, 339693067, 339695054, 339698714, 339699319, 339701747, 339703698, 339705531, 339709440, 339710854, 339712374, 339714992, 339720853, 339721992, 339728838, 339729335, 339729940, 339731500, 339742772, 339747690, 339748485, 339750356, 339767823, 339769222, 339770160, 339773211, 339774304, 339780706, 339782155, 339787531, 339815496, 339831814, 339849953, 339850

In [ ]:
# [https://vk.com/kazan.club] 85479
p = getFromGroup(gid='int_kazan', offset = 0, members_count = 85479)
print('len p = ', len(p))
print(p)

In [ ]:
# [https://vk.com/kzngot] 51993
p = getFromGroup(gid='int_kazan', offset = 0, members_count = 51993)
print('len p = ', len(p))
print(p)